In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf

In [7]:

# training = pd.read_csv("./data/train.csv")

In [8]:
#import mnist dataset
# from tensorflow.examples.tutorials.mnist import input_data
# mnist=input_data.read_data_sets("/tmp/.data/",one_hot=True)
print(chr(65))

A


In [12]:
class DataLoader:
    def __init__(self):
        self.i = 0
        self.char_to_ind = None
        self.ind_to_char = None
        
        self.data = self.loadData()
        
    def loadData(self):
        with zipfile.ZipFile("alllines.txt.zip") as zip_file:
            with zip_file.open(zip_file.namelist()[0]) as text_file:
                data = text_file.read()
        data = data.decode("UTF-8").replace("\"", "")

        return list(data)
    
    def getData(self):
        return self.data
    
    
    def nextBatch(self, batch_size=128, step_size=10):
        
        c2i, _ = self.getMappings()
        
        
        x = np.zeros((batch_size, step_size, len(c2i)))
        y = np.zeros((batch_size,len(c2i)))
        for batch_num in range(batch_size):
            x_t, y_t = self.next_time_step(step_size)
            for step in range(step_size):
                i, j, k = batch_num, step, c2i[x_t[step]]
                x[i, j, k] = 1.0
            try:
                y[i, c2i[y_t[step]]] = 1.0
            except Exception as e:
                print("len: {}, step: {}".format(len(y_t), step))
                raise e
        
        return x, y
        
    def next_time_step(self, step_size = 128):
        x, y = None, None
        if self.i  + step_size + 1 >= len(self.data):
            assert len(self.data) > self.i
            remaining = step_size - (len(self.data) - self.i)
            x = self.data[self.i:] + self.data[:remaining + 1]
            y = self.data[self.i + 1: ] + self.data[1:remaining + 2]
            self.i = remaining
        else:
            x = self.data[self.i:self.i + step_size]
            y = self.data[self.i + 1:self.i + step_size + 1]
            self.i += step_size

        return x, y
    
    def getMappings(self):
        if self.char_to_ind and self.ind_to_char:
            return self.char_to_ind, self.ind_to_char
        
        chars = [c for c in set(self.data)]
        char_to_ind, ind_to_char = {}, {}
        
        index = 0
        for char in chars:
            char_to_ind[char] = index
            ind_to_char[index] = char
            index += 1
        
        self.char_to_ind = char_to_ind
        self.ind_to_char = ind_to_char
        
        return char_to_ind, ind_to_char
print("done")
# zzz = DataLoader()
# zzz.nextBatch(batch_size=300, step_size=20)

done


In [13]:
input_data = DataLoader()
c2i, i2c = input_data.getMappings()

# ltsm setup
tf.reset_default_graph()
#f.variables
time_steps = 100
ltsm_cells = 128
n_input = len(c2i)
learning_rate = 0.001
n_classes = len(c2i)
batch_size = 1000

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([ltsm_cells,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

x = tf.placeholder(tf.float32, [None, time_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

inp = tf.unstack(x, time_steps, 1)

ltsm_layer = tf.contrib.rnn.BasicLSTMCell(ltsm_cells, forget_bias=1)
outputs, _ = tf.contrib.rnn.static_rnn(ltsm_layer, inp, dtype=tf.float32)

prediction = tf.matmul(outputs[-1], out_weights) + out_bias

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [ ]:

with tf.Session() as sess:
    tf.global_variables_initializer().run(session=sess)
    

    for i in range(800):
        batch_x, batch_y = input_data.nextBatch(batch_size=batch_size, step_size=time_steps)
#         batch_x = batch_x.reshape((batch_size, time_steps, n_input))
        
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if i %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("step: {}, acc: {:.2f}%, loss: {:.2f}".format(i, acc * 100, los))
            
    
    #calculating test accuracy
#     test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
#     test_label = mnist.test.labels[:128]
#     print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

step: 0, acc: 2.70%, loss: 4.68
step: 10, acc: 14.20%, loss: 3.58
step: 20, acc: 16.00%, loss: 3.35
step: 30, acc: 16.80%, loss: 3.17
step: 40, acc: 15.10%, loss: 3.11
step: 50, acc: 20.50%, loss: 2.94
step: 60, acc: 22.20%, loss: 2.92
step: 70, acc: 22.50%, loss: 2.88
step: 80, acc: 24.60%, loss: 2.83
